In [16]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime as dt
import pyrealsense2 as rs

In [24]:
width, height, framerate = (640, 480, 30)
pipeline = rs.pipeline()
config = rs.config()

config.enable_stream(rs.stream.depth, 
    width, height, rs.format.z16, framerate)
config.enable_stream(rs.stream.color, 
    width, height, rs.format.bgr8, framerate)

th_filter = rs.threshold_filter(max_dist=2.5)
sp_filter = rs.spatial_filter()
sp_filter.set_option(rs.option.filter_magnitude, 3.0)
sp_filter.set_option(rs.option.holes_fill, 2.0)
tmp_filter = rs.temporal_filter()
pc = rs.pointcloud()


# Start stream
pipeline.start(config)


In [26]:
try:
    while True:
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        # filter
        depth_frame = th_filter.process(depth_frame)
        depth_frame = sp_filter.process(depth_frame)
        depth_frame = tmp_filter.process(depth_frame)

        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(
            depth_image, alpha=0.1), cv2.COLORMAP_JET)
        images = np.hstack((color_image, depth_colormap))
        
        
        
        cv2.namedWindow("rs", cv2.WINDOW_AUTOSIZE)
        cv2.imshow("rs", images)
        cv2.waitKey(1)
        
finally:
    pc.map_to(depth_frame)
    points = pc.calculate(depth_frame)
    points.export_to_ply("./out.ply", color_frame)
    pipeline.stop()
    cv2.destroyAllWindows()

KeyboardInterrupt: 